In [1]:
import os.path
from sklearn.feature_extraction.text import CountVectorizer
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
from standard_function import *
from return_detail_icd_code import *
import pickle
import unicodedata
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import pandas as pd

In [2]:
load_data_check_default = pd.read_excel("dataset_train.xls", header = None, encoding = 'utf-8')
load_data_check = load_data_check_default.copy()
load_data_check[2] = load_data_check[2].apply(delete_special_char)

#test_dataframe = pd.read_excel("AHUY_1406.xls")
test_data_col = load_data_check[2]
test_data_col.head()

0                                              bệnh tả
1                                              bệnh tả
2    bệnh tả do vibrio cholerae 01  typ sinh học ch...
3    bệnh tả do vibrio cholerae 01  typ sinh học eltor
4                    bệnh thương hàn và phó thương hàn
Name: 2, dtype: object

In [5]:


f = open('Eclaim_predict.pickle', 'rb')       # load model dự đoán mã icd 
classifier = pickle.load(f)
f.close()

with open ('exacly_2_word', 'rb') as fp_2:    # Load data về 2 cụm từ 
    words_2 = pickle.load(fp_2)
    
with open ('exacly_3_word', 'rb') as fp_3:    # Load data về 3 cụm từ
    words_3 = pickle.load(fp_3)
    
with open('link_words.pkl', 'rb') as f:
    listview = pickle.load(f)
    
model_2 = Word2Vec.load("2_gram.model")
model_3 = Word2Vec.load("3_gram.model")

def try_except_2_gram ( string):
    try :  
        model_2.wv.similar_by_word(string)
    except :
        kq = False
    else :
        kq = True
    return kq

def try_except_3_gram ( string):
    try :  
        model_3.wv.similar_by_word(string)
    except :
        kq = False
    else :
        kq = True
    return kq

dim_test_col = len(test_data_col)
var_run = 0
result_code = []

print("Running..........................!")

while var_run < dim_test_col:
    
    text = test_data_col[var_run]

    text = revert_text(out_special_char(text))    # hàm loại bỏ dấu ngoặc tròn, vuông , \ , /
    pr = spaces_num(char_num(text))    # loại bỏ khoản trắng vô nghĩa  
    pr1 = ' '.join(pr)                 # convert  list to string
    pr1 = delete_special_char(pr1)     # delete others special char --- > string return str

    text1 = pr1.strip("").split(" ")       # Convert str to list
    text1 = spaces_num(text1)                # loại bỏ khoản trắng thừa trong list
    text2 = pr1.strip("").split("*")       # Convert str to list Text 1 chứa list các từ đơn
    text2 = spaces_num(text2)                # Text2 chứa List cả cụm từ
    temp_02 = text1.copy()

    check_logic = True
    if (len(text1) >= 3):
        sta = incorrect_spelling(text1, words_2, words_3)
        text1 = sta.strip(' ').split(' ')
        text2 = sta.strip(' ').split('*')
    elif len(text1) == 2:
        kq = check_incorrect_2_word(text1,words_2)
        text1 = kq[0]
        check_logic = kq[1]
    if len(text1) == 1:
        pass

    new_str =''
    for i in range(0,len(text1)-1):
        new_str += text1[i] + ' ' + text1[i+1] + ","

    new_str = new_str.strip(',').split(",")
    addr = []
    chuoi = ''
    for index,i in enumerate (new_str[:len(new_str)]):
        if try_except_2_gram(i) == False :
            addr.append(index)
            chuoi += i + ","
    if len(text1)-len(addr) == 1:
        addr.append(max(addr) + 1)

    if len(addr) == 0:
        addr.append(-1)
    sum_index = addr[0]
    temp = []
    for i in range(0,len(addr)-1):
        if (addr[i+1] - addr[i]) == 1:
            sum_index = addr[i+1]
        else:
            temp.append(sum_index)
            sum_index = addr[i+1]
    temp.append(sum_index)
    if temp[0] == '':
        temp.append(len(text1))
    else:
        temp = list(set(temp))
        temp.sort(reverse = False)

    check = ''
    emty = ''
    sta = []
    sto = []
    sta1 = []
    for index, x in enumerate(text1):     # cắt chuỗi bệnh thành các loại bệnh bước 1
        if index in temp:
            check += x + ','
        else:
            check += x + ' '
    check = spaces_num(check.strip().split(','))

    if check_logic == False:             # Tách 2 bệnh chỉ có 2 chữ cái
        check = temp_02

    for i in range(0, len(check)):       # Gộp các từ đơn lẻ tránh lỗi Vocabulary
        if len(check[i]) == 1:
            check.remove(check[i])
            break

    for i in range(0, len(check)): 
        if len(check[i]) == 1:
            check[i-1] = check[i-1] + ' ' + check[i]
            check.remove(check[i])

    for i in check:                     # Tách các bệnh dính liền thành các loại bệnh bước 2
        ix = i.split(' ')
        for j in listview:
            if j in i:
                jx = j.split(' ')
                for index, val in enumerate(ix[0:len(ix)-2]):
                    if (ix[index] == jx[0]) and (ix[index+1]==jx[1]) and (try_except_3_gram(j+' '+ix[index+2]) == False):
                        ix.insert(index+1, ',') 
        ix = ' '.join(ix)
        i = ix
        emty += i +','
    check = emty.strip(',').split(',')

    with open ('sample_vector', 'rb') as fp:
        itemlist = pickle.load(fp)

    count = CountVectorizer()
    count.fit(itemlist)
    bag_of_words = count.transform(itemlist)

    x_new1 = count.transform(check)
    x_new1
    kq = classifier.predict(x_new1)
    h =1
    dicti = []
    chieu_dai = len(kq)
    for g in range(0,chieu_dai):
        df_end = (load_data_check[load_data_check[1] == kq[g]])
        io = decission_value_return_01(df_end, check[g].split(','))
        
       #if io[1]<0.9:
        Sam2Data = io[0].strip() + ' ' + str(io[1])
        dicti.append(Sam2Data)
        #else : pass
    #if dicti!=[]:
    print(var_run+1,dicti,',')
    #else : pass
    Sam2Data=''
    dicti = []
    var_run += 1
print("Completed !")

Running..........................!
1 ['A00 1.0'] ,
2 ['A00 1.0'] ,
3 ['A001 0.87', 'P61 0.53'] ,
4 ['A001 0.87', 'P61 0.49'] ,
5 ['A01 0.85'] ,
6 ['A010 1.0'] ,
7 ['A011 1.0'] ,
8 ['A012 1.0'] ,
9 ['A013 1.0'] ,
10 ['A014 0.99'] ,
11 ['A02 1.0'] ,
12 ['A020 1.0'] ,
13 ['A021 1.0'] ,
14 ['A022 1.0'] ,
15 ['A028 1.0'] ,
16 ['A029 0.99'] ,
17 ['A03 1.0'] ,
18 ['A03 1.0'] ,
19 ['A03 1.0'] ,
20 ['A03 1.0'] ,
21 ['A030 1.0'] ,
22 ['A031 1.0'] ,
23 ['A032 1.0'] ,
24 ['A033 1.0'] ,
25 ['A038 1.0'] ,
26 ['A039 0.99'] ,
27 ['A04 1.0'] ,
28 ['A040 1.0'] ,
29 ['A041 1.0'] ,
30 ['A042 1.0'] ,
31 ['A043 1.0'] ,
32 ['A044 1.0'] ,
33 ['A045 1.0'] ,
34 ['A046 1.0'] ,
35 ['A047 1.0'] ,
36 ['A048 1.0'] ,
37 ['A049 0.99'] ,
38 ['A05 0.96'] ,
39 ['A050 0.95'] ,
40 ['A051 0.86'] ,
41 ['A052 0.91'] ,
42 ['A053 0.95'] ,
43 ['A054 0.95'] ,
44 ['A058 0.96'] ,
45 ['A059 0.99'] ,
46 ['A06 1.0'] ,
47 ['A060 1.0'] ,
48 ['A061 1.0'] ,
49 ['A062 1.0'] ,
50 ['A063 1.0'] ,
51 ['A064 1.0'] ,
52 ['A065 1.0'] ,
53 ['A066 

434 ['A740 1.0'] ,
435 ['A740 1.0'] ,
436 ['A748 1.0'] ,
437 ['A749 0.99'] ,
438 ['A75 1.0'] ,
439 ['A750 1.0'] ,
440 ['A751 0.99'] ,
441 ['A752 1.0'] ,
442 ['A753 1.0'] ,
443 ['A759 0.99'] ,
444 ['A77 0.99'] ,
445 ['A770 1.0'] ,
446 ['A771 1.0'] ,
447 ['A772 1.0'] ,
448 ['A773 1.0'] ,
449 ['A778 1.0'] ,
450 ['A779 0.99'] ,
451 ['A79 1.0'] ,
452 ['A791 0.9'] ,
453 ['A798 1.0'] ,
454 ['A799 0.99'] ,
455 ['A799 0.99'] ,
456 ['A80 1.0'] ,
457 ['A800 0.94'] ,
458 ['A801 0.95'] ,
459 ['A802 0.94'] ,
460 ['A803 0.95'] ,
461 ['A804 0.91'] ,
462 ['A809 0.99'] ,
463 ['A81 1.0'] ,
464 ['A810 0.99'] ,
465 ['A811 1.0'] ,
466 ['A812 1.0'] ,
467 ['A818 1.0'] ,
468 ['A819 0.96'] ,
469 ['A82 1.0'] ,
470 ['A82 1.0'] ,
471 ['A820 1.0'] ,
472 ['A821 1.0'] ,
473 ['A829 0.99'] ,
474 ['A83 1.0'] ,
475 ['A830 1.0'] ,
476 ['A831 1.0'] ,
477 ['A832 1.0'] ,
478 ['A833 0.98'] ,
479 ['A834 1.0'] ,
480 ['A835 1.0'] ,
481 ['A836 1.0'] ,
482 ['A838 1.0'] ,
483 ['A839 0.99'] ,
484 ['A84 1.0'] ,
485 ['A840 0.91'] ,
48

866 ['B559 0.99'] ,
867 ['B56 1.0'] ,
868 ['B56 1.0'] ,
869 ['B560 1.0'] ,
870 ['B561 1.0'] ,
871 ['B569 0.99'] ,
872 ['B57 1.0'] ,
873 ['B570 1.0'] ,
874 ['B571 1.0'] ,
875 ['B572 0.98'] ,
876 ['B573 0.99'] ,
877 ['B574 0.99'] ,
878 ['B575 0.99'] ,
879 ['B58 1.0'] ,
880 ['B580 1.0'] ,
881 ['B581 1.0'] ,
882 ['B582 0.92'] ,
883 ['B583 1.0'] ,
884 ['B588 1.0'] ,
885 ['B589 0.99'] ,
886 ['B59 1.0'] ,
887 ['B59 1.0'] ,
888 ['B59 1.0'] ,
889 ['B59 1.0'] ,
890 ['B59 1.0'] ,
891 ['B60 0.99'] ,
892 ['B600 1.0'] ,
893 ['B601 1.0'] ,
894 ['B602 1.0'] ,
895 ['B608 0.99'] ,
896 ['B64 1.0'] ,
897 ['B64 1.0'] ,
898 ['B64 1.0'] ,
899 ['B64 1.0'] ,
900 ['B64 1.0'] ,
901 ['B65 0.99'] ,
902 ['B650 0.87'] ,
903 ['B651 0.86'] ,
904 ['B652 1.0'] ,
905 ['B653 1.0'] ,
906 ['B658 1.0'] ,
907 ['B659 0.99'] ,
908 ['B66 1.0'] ,
909 ['B660 1.0'] ,
910 ['B661 1.0'] ,
911 ['B663 1.0'] ,
912 ['B663 1.0'] ,
913 ['B665 1.0'] ,
914 ['B488 0.77'] ,
915 ['B669 0.99'] ,
916 ['B67 1.0'] ,
917 ['B670 1.0'] ,
918 ['B671 1.0

1285 ['C26 1.0'] ,
1286 ['C260 1.0'] ,
1287 ['C261 1.0'] ,
1288 ['C261 1.0'] ,
1289 ['C261 1.0'] ,
1290 ['C728 0.73'] ,
1291 ['C269 1.0'] ,
1292 ['C30 1.0'] ,
1293 ['C300 1.0'] ,
1294 ['C301 1.0'] ,
1295 ['C300 1.0'] ,
1296 ['C301 1.0'] ,
1297 ['C31 1.0'] ,
1298 ['C310 1.0'] ,
1299 ['C311 1.0'] ,
1300 ['C312 1.0'] ,
1301 ['C313 1.0'] ,
1302 ['C318 1.0'] ,
1303 ['C319 0.99'] ,
1304 ['C32 1.0'] ,
1305 ['C320 1.0'] ,
1306 ['C321 1.0'] ,
1307 ['C322 1.0'] ,
1308 ['C323 1.0'] ,
1309 ['C348 0.88'] ,
1310 ['C329 0.99'] ,
1311 ['C33 1.0'] ,
1312 ['C33 1.0'] ,
1313 ['C33 1.0'] ,
1314 ['C33 1.0'] ,
1315 ['C33 1.0'] ,
1316 ['C34 1.0'] ,
1317 ['C34 1.0'] ,
1318 ['C340 1.0'] ,
1319 ['C341 0.99'] ,
1320 ['C342 0.99'] ,
1321 ['C343 0.99'] ,
1322 ['C348 1.0'] ,
1323 ['C349 0.99'] ,
1324 ['C37 1.0'] ,
1325 ['C37 1.0'] ,
1326 ['C37 1.0'] ,
1327 ['C37 1.0'] ,
1328 ['C37 1.0'] ,
1329 ['C38 0.99'] ,
1330 ['C380 1.0'] ,
1331 ['C381 1.0'] ,
1332 ['C382 1.0'] ,
1333 ['C383 0.99'] ,
1334 ['C384 1.0'] ,
1335 ['

1693 ['C880 1.0'] ,
1694 ['C881 1.0'] ,
1695 ['C882 1.0'] ,
1696 ['C883 1.0'] ,
1697 ['C887 1.0'] ,
1698 ['C889 1.0'] ,
1699 ['C90 0.94'] ,
1700 ['C900 1.0'] ,
1701 ['C901 1.0'] ,
1702 ['C902 1.0'] ,
1703 ['C903 1.0'] ,
1704 ['C91 1.0'] ,
1705 ['C910 0.99'] ,
1706 ['C911 1.0'] ,
1707 ['C912 1.0'] ,
1708 ['C913 1.0'] ,
1709 ['C914 1.0'] ,
1710 ['C915 1.0'] ,
1711 ['C916 1.0'] ,
1712 ['C917 1.0'] ,
1713 ['C918 1.0'] ,
1714 ['C919 0.99'] ,
1715 ['C92 1.0'] ,
1716 ['C920 1.0'] ,
1717 ['C921 0.98'] ,
1718 ['C922 0.96'] ,
1719 ['C46 0.72'] ,
1720 ['C924 0.99'] ,
1721 ['C931 0.79'] ,
1722 ['C926 0.97'] ,
1723 ['C927 1.0'] ,
1724 ['C928 1.0'] ,
1725 ['C929 0.99'] ,
1726 ['C93 1.0'] ,
1727 ['C930 0.91'] ,
1728 ['C931 1.0'] ,
1729 ['C932 1.0'] ,
1730 ['C933 1.0'] ,
1731 ['C937 1.0'] ,
1732 ['C939 1.0'] ,
1733 ['C957 0.91'] ,
1734 ['C940 0.93'] ,
1735 ['C941 1.0'] ,
1736 ['C942 0.95'] ,
1737 ['Q822 1.0'] ,
1738 ['C943 1.0'] ,
1739 ['C944 0.94'] ,
1740 ['C945 1.0'] ,
1741 ['C946 0.96'] ,
1742 ['C9

...

9851 ['R99 0.88'] ,
9852 ['R99 0.88'] ,
9853 ['S00 1.0'] ,
9854 ['S000 1.0'] ,
9855 ['S001 0.95'] ,
9856 ['S002 0.96'] ,
9857 ['S003 1.0'] ,
9858 ['S004 1.0'] ,
9859 ['S005 1.0'] ,
9860 ['S007 1.0'] ,
9861 ['S008 0.94'] ,
9862 ['S009 0.99'] ,
9863 ['S01 1.0'] ,
9864 ['S01 1.0'] ,
9865 ['S01 1.0'] ,
9866 ['S01 1.0'] ,
9867 ['S01 1.0'] ,
9868 ['S010 1.0'] ,
9869 ['S011 0.95'] ,
9870 ['S012 1.0'] ,
9871 ['S013 1.0'] ,
9872 ['S014 0.99'] ,
9873 ['S014 0.93'] ,
9874 ['S015 1.0'] ,
9875 ['S015 1.0'] ,
9876 ['S015 1.0'] ,
9877 ['S017 1.0'] ,
9878 ['S018 0.95'] ,
9879 ['S019 0.99'] ,
9880 ['S02 0.92'] ,
9881 ['S020 1.0'] ,
9882 ['S021 1.0'] ,
9883 ['S022 1.0'] ,
9884 ['S023 1.0'] ,
9885 ['S024 0.92'] ,
9886 ['K031 0.56'] ,
9887 ['S026 1.0'] ,
9888 ['S027 0.95'] ,
9889 ['S028 1.0'] ,
9890 ['S029 0.95'] ,
9891 ['S03 0.9'] ,
9892 ['S030 1.0'] ,
9893 ['S031 1.0'] ,
9894 ['S032 1.0'] ,
9895 ['S033 0.95'] ,
9896 ['S034 1.0'] ,
9897 ['S035 0.92'] ,
9898 ['S04 1.0'] ,
9899 ['S040 1.0'] ,
9900 ['S041 1

10247 ['S37 1.0'] ,
10248 ['S370 1.0'] ,
10249 ['S371 1.0'] ,
10250 ['S372 0.99'] ,
10251 ['S372 0.99'] ,
10252 ['S373 1.0'] ,
10253 ['S374 1.0'] ,
10254 ['S375 1.0'] ,
10255 ['S376 1.0'] ,
10256 ['S377 1.0'] ,
10257 ['S378 1.0'] ,
10258 ['S379 1.0'] ,
10259 ['S38 0.96'] ,
10260 ['S380 1.0'] ,
10261 ['S381 0.99'] ,
10262 ['S382 1.0'] ,
10263 ['S383 0.96'] ,
10264 ['S39 0.96'] ,
10265 ['S390 0.95'] ,
10266 ['S396 0.86'] ,
10267 ['S397 0.99'] ,
10268 ['S398 0.99'] ,
10269 ['S399 0.99'] ,
10270 ['S40 1.0'] ,
10271 ['S400 1.0'] ,
10272 ['S407 1.0'] ,
10273 ['S408 1.0'] ,
10274 ['S409 0.99'] ,
10275 ['S41 1.0'] ,
10276 ['S410 1.0'] ,
10277 ['S411 1.0'] ,
10278 ['S417 1.0'] ,
10279 ['S318 0.74'] ,
10280 ['S42 0.93'] ,
10281 ['S420 1.0'] ,
10282 ['S420 1.0'] ,
10283 ['S421 1.0'] ,
10284 ['S422 1.0'] ,
10285 ['S423 1.0'] ,
10286 ['S424 1.0'] ,
10287 ['S427 0.91'] ,
10288 ['S428 0.96'] ,
10289 ['S429 0.99'] ,
10290 ['S43 0.92'] ,
10291 ['S430 1.0'] ,
10292 ['S431 1.0'] ,
10293 ['S432 1.0'] ,
10

10633 ['S913 1.0'] ,
10634 ['S917 1.0'] ,
10635 ['S92 0.92'] ,
10636 ['S920 1.0'] ,
10637 ['S920 1.0'] ,
10638 ['S920 1.0'] ,
10639 ['S826 0.89'] ,
10640 ['S827 0.66'] ,
10641 ['S923 1.0'] ,
10642 ['S924 1.0'] ,
10643 ['S925 1.0'] ,
10644 ['S927 1.0'] ,
10645 ['S929 1.0'] ,
10646 ['S93 0.91'] ,
10647 ['S930 1.0'] ,
10648 ['S931 1.0'] ,
10649 ['S932 1.0'] ,
10650 ['S932 0.95'] ,
10651 ['S933 1.0'] ,
10652 ['S934 1.0'] ,
10653 ['S935 1.0'] ,
10654 ['S936 0.99'] ,
10655 ['S95 0.96'] ,
10656 ['S950 1.0'] ,
10657 ['S951 1.0'] ,
10658 ['S952 1.0'] ,
10659 ['S957 0.96'] ,
10660 ['S958 0.96'] ,
10661 ['S959 0.96'] ,
10662 ['S96 0.93'] ,
10663 ['S960 0.91'] ,
10664 ['S961 0.91'] ,
10665 ['S962 0.93'] ,
10666 ['S967 0.93'] ,
10667 ['S968 0.93'] ,
10668 ['S969 0.93'] ,
10669 ['S99 0.93'] ,
10670 ['S998 0.96'] ,
10671 ['S999 0.96'] ,
10672 ['S509 0.82'] ,
10673 ['T01 1.0'] ,
10674 ['T010 1.0'] ,
10675 ['T011 0.99'] ,
10676 ['T012 1.0'] ,
10677 ['T013 0.96'] ,
10678 ['T016 0.97'] ,
10679 ['T018 1.0

11007 ['T408 0.99'] ,
11008 ['T409 0.91'] ,
11009 ['T41 1.0'] ,
11010 ['T410 1.0'] ,
11011 ['T411 1.0'] ,
11012 ['T412 1.0'] ,
11013 ['T413 1.0'] ,
11014 ['T414 0.99'] ,
11015 ['T415 1.0'] ,
11016 ['T42 0.99'] ,
11017 ['T420 1.0'] ,
11018 ['T610 0.53'] ,
11019 ['T422 1.0'] ,
11020 ['T423 1.0'] ,
11021 ['T424 1.0'] ,
11022 ['T425 0.99'] ,
11023 ['T426 1.0'] ,
11024 ['T427 0.99'] ,
11025 ['T428 0.97'] ,
11026 ['T43 0.99'] ,
11027 ['T430 0.96'] ,
11028 ['T431 1.0'] ,
11029 ['T432 1.0'] ,
11030 ['T433 0.9'] ,
11031 ['T434 1.0'] ,
11032 ['T435 0.94'] ,
11033 ['T436 0.96'] ,
11034 ['T442 0.71'] ,
11035 ['T439 0.99'] ,
11036 ['T44 0.88'] ,
11037 ['T440 1.0'] ,
11038 ['T441 1.0'] ,
11039 ['T442 0.99'] ,
11040 ['T443 0.92'] ,
11041 ['T444 0.99'] ,
11042 ['T445 0.99'] ,
11043 ['T446 0.99'] ,
11044 ['T447 0.99'] ,
11045 ['T448 0.97'] ,
11046 ['T449 0.97'] ,
11047 ['T45 0.99'] ,
11048 ['T450 0.95'] ,
11049 ['T451 1.0'] ,
11050 ['T452 0.99'] ,
11051 ['T453 0.99'] ,
11052 ['T385 0.54'] ,
11053 ['T45

11394 ['T884 1.0'] ,
11395 ['T885 1.0'] ,
11396 ['T780 0.46'] ,
11397 ['T886 1.0'] ,
11398 ['T789 0.61'] ,
11399 ['T888 1.0'] ,
11400 ['T889 0.99'] ,
11401 ['T90 1.0'] ,
11402 ['T900 1.0'] ,
11403 ['T901 1.0'] ,
11404 ['T902 0.94'] ,
11405 ['T924 0.77'] ,
11406 ['T904 0.95'] ,
11407 ['T905 1.0'] ,
11408 ['T908 1.0'] ,
11409 ['T909 1.0'] ,
11410 ['T91 1.0'] ,
11411 ['T910 0.93'] ,
11412 ['T911 1.0'] ,
11413 ['T912 1.0'] ,
11414 ['T913 1.0'] ,
11415 ['T914 1.0'] ,
11416 ['T915 1.0'] ,
11417 ['T918 1.0'] ,
11418 ['T919 1.0'] ,
11419 ['T92 1.0'] ,
11420 ['T930 0.86'] ,
11421 ['T921 1.0'] ,
11422 ['S62 0.78'] ,
11423 ['T039 0.63'] ,
11424 ['T924 1.0'] ,
11425 ['T925 1.0'] ,
11426 ['T926 0.96'] ,
11427 ['T936 0.96'] ,
11428 ['T928 1.0'] ,
11429 ['T929 1.0'] ,
11430 ['T93 1.0'] ,
11431 ['T930 1.0'] ,
11432 ['T931 1.0'] ,
11433 ['T932 1.0'] ,
11434 ['T039 0.62'] ,
11435 ['T934 1.0'] ,
11436 ['T935 1.0'] ,
11437 ['T938 1.0'] ,
11438 ['T939 1.0'] ,
11439 ['T94 1.0'] ,
11440 ['T940 1.0'] ,
11441 